# Eval IoU of Segmentation model with different inpul image angle for the rotation equivariance model

In [9]:
%matplotlib inline
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models
import torchvision.transforms as transforms
import torchvision.transforms.functional as TF
from torch.utils.data import DataLoader, Dataset
import matplotlib.pyplot as plt
import sys
sys.path.insert(1, '../../utils')
sys.path.insert(1, '../../datasets')
sys.path.insert(1, '../../models')
from utils import * 
import eval_train as ev
import my_datasets as mdset
from fully_supervised_models import Fully_Supervised_SegNet

In [4]:
PATH_MODEL = '/users/k/karmimy/Recherche/Equivariance/equivariance_model/models/lightning_logs/version_0/checkpoints/epoch=4.ckpt'
dataroot_voc = '~/data/voc2012'


In [5]:
batch_size = 2
rotate = False
criterion_unsupervised = nn.CrossEntropyLoss(ignore_index=21)
#criterion_supervised = nn.CrossEntropyLoss(ignore_index=21) # On ignore la classe border.
Loss = 'CE' # Loss = 'KL' or 'CE' or None for L1,MSE…
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device :",device)

device : cuda:0


In [7]:
train_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='train', download=True,rotate=rotate)
val_dataset_VOC = mdset.VOCSegmentation(dataroot_voc,year='2012', image_set='val', download=True)

Using downloaded and verified file: /users/k/karmimy/data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /users/k/karmimy/data/voc2012/VOCtrainval_11-May-2012.tar


In [8]:
dataloader_train_VOC = DataLoader(train_dataset_VOC, batch_size=batch_size,shuffle=True,drop_last=True)
dataloader_val = DataLoader(val_dataset_VOC, batch_size=batch_size)

## Load model

In [7]:
def load_model(file=None,fcn=False):
    if file is None:
        if fcn is False:
            model = torchvision.models.segmentation.deeplabv3_resnet101(pretrained=True)
        else:
            model = torchvision.models.segmentation.fcn_resnet101(pretrained=True)
    else:
        model = torch.load(os.path.join(SAVE_DIR,file))
    return model

In [10]:
model = Fully_Supervised_SegNet.load_from_checkpoint(checkpoint_path=PATH_MODEL)

IsADirectoryError: [Errno 21] Is a directory: '/users/k/karmimy/Recherche/Equivariance/equivariance_model/models/lightning_logs/version_0/checkpoints'

In [9]:
model.eval()
print("EVAL ON TRAIN VOC 2012:")
m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_train_VOC,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
d_iou = ev.eval_model_all_angle(model,train=True,batch_size=batch_size,device=device,num_classes=21)
print("EVAL ON VAL VOC 2012:")
m_pix_acc, m_loss_equiv = eval_accuracy_equiv(model,dataloader_val,criterion=criterion_unsupervised,\
                        nclass=21,device=device,Loss=Loss,plot=False,angle_max=30,random_angle=False) 
d_iou = ev.eval_model_all_angle(model,train=False,batch_size=batch_size,device=device,num_classes=21)

EVAL ON TRAIN VOC 2012:


/workspace/utils.py:143: RuntimeWarning: invalid value encountered in true_divide
  acc_cls = np.diag(hist) / hist.sum(axis=1)
/workspace/utils.py:145: RuntimeWarning: invalid value encountered in true_divide
  iu = np.diag(hist) / (hist.sum(axis=1) + hist.sum(axis=0) - np.diag(hist))


Mean Pixel Accuracy between masks : 0.8948315740977913 Loss Validation : 0.32087162
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Using downloaded and verified file: /data/voc2012/VOCtrainval_11-May-2012.tar
Scores for datasets rotate by 0 degrees:
   mIoU 0.5548123088165193 Accuracy 0.8466233514606978 CE Loss 0.5434119888311608
Scores for datasets rotate by 10 degrees:
   mIoU 0.5268096251539204 Accuracy 0.8413524766991626 CE Loss 0.5604919073416195
Scores for datasets rotate by 20 degrees:
   mIoU 0.4935260467460531 Accuracy 0.8317718084117438 CE Loss 0.5871318546658169